In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import shutil
import json
import pickle
from tqdm import tqdm
import random
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image

from load_model import load_model
from load_model_barlowtwins import load_barlowtwins_model

from rn50_auxiliary_dm import rn50_auxiliary_dm

import scipy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [ ]:
# Experiment list summarized

# Basic GFR/CTFR/GOR tasks 
exp_names_list_basics = [
    # 1. (AppleMesh09) Apple GFR - AppleMesh00/AppleMesh01 neutral
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_AppleMesh00',
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_AppleMesh01',

    # 2. (AppleMesh02) Apple GFR - AppleMesh00/AppleMesh01 neutral
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral',

    # 3. (AppleMesh09) Apple GFR - AppleMesh03/Praneeth
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_AppleMesh03_praneeth/20230406_Var6vbsl_set0_im151_praneeth_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_AppleMesh03_praneeth/20230406_Var6vbsl_set0_im151_AppleMesh03_dur200ms_lab',

    # 4. (AppleMesh09) Basel CTFR 
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_dur200ms_lab',

    # 5. (AppleMesh09) Basel GFR
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_notexture_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_notexture_dur200ms_lab',
]

# complete GFR
exp_names_list_GFR = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral',
]

# complete CTFR
exp_names_list_CTFR = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh01_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh02_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh03_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh04_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh05_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh06_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh07_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh08_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh09_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh10_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh11_neutral',
]

# complete GTFR 
exp_names_list_GTFR = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh01_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh02_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh03_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh04_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh05_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh06_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh07_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh08_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh09_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh10_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh11_neutral',
]

# complete GER (AppleMesh03)
exp_names_list_GER_AppleMesh03 = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/vbsle151_AppleMesh03_surprise_4',
   ]

# complete GER (AppleMesh00)
exp_names_list_GER_AppleMesh00  = [ 
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/vbsle151_AppleMesh00_surprise_4',
    ]

# complete CTER (AppleMesh00)
exp_names_list_CTER_AppleMesh00 = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_AppleMesh00_surprise_4',
]

# complete GTER (AppleMesh00)
exp_names_list_GTER_AppleMesh00 = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_disgust_4'
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_AppleMesh00_surprise_4',
]

# complete Basel GFR (id10-21)
exp_names_list_basel_GFR = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id10',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id11',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id12',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id13',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id14',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id15',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id16',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id17',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id18',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id19',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id20',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id21',
]

# complte Basel CTFR (id10-21)
exp_names_list_basel_CTFR = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id10',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id11',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id12',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id13',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id14',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id15',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id16',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id17',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id18',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id19',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id20',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id21',
]

# complete Basel GFR (id988-999)
exp_names_list_basel_GFR_ver2 = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id988',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id989',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id990',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id991',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id992',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id993',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id994',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id995',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id996',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id997',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id998',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id999',
]

# complte Basel CTFR (id988-999)
exp_names_list_basel_CTFR_ver2 = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id988',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id989',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id990',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id991',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id992',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id993',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id994',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id995',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id996',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id997',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id998',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id999',
]

# LFW (upright, inverted)
exp_names_list_lfw = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell/Bush',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell/Powell',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell_inverted/Bush',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell_inverted/Powell',
]

# YouTube Faces
exp_names_list_ytface = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Stanca',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Beard',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Kennedy',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Macdonald',
]

# complete GOR 
exp_names_list_GOR = [
    # 1. (AppleMesh09) GOR - Camel/Elephant
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_camel',
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant',

    # 2. GOR - Dog/Horse 
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dog_horse/vbsle_151_dog', 
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dog_horse/vbsle_151_horse',

    # 3. GTOR - Camel/Elephant
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_gray_downsampled/camel_colored_gray',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_gray_downsampled/elephant_colored_gray',
    
    # 4. CTOR - Camel/Elephant
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_downsampled/camel_colored',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_downsampled/elephant_colored',
]

# ImageNet-based tasks (general object recognition)
exp_names_list_imagenet = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/shark151/shark1',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/shark151/shark2',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/bird151/bird1',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/bird151/bird2',
    
    '/mnt/smb/locker/issa-locker/users/Seojin/data/vehicle151/bike',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/vehicle151/car'
]

# extract feats

In [ ]:
# Extract feautres for standard models 

# Specify experiment type 
exp_names_list = exp_names_list_GFR

# Specify the list of "model names"
model_names_list=[
# 'off_the_shelf_barlowtwins_finetune_7way_EM_AppleMesh03_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_7way_EM_AppleMesh04_colorbg_seed77__model_best',
# 'off_the_shelf_barlowtwins_finetune_7way_EM_AppleMesh08_colorbg_seed77__model_best',
'off_the_shelf_barlowtwins_finetune_4way_IDEM_AppleMesh02_AppleMesh03_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_4way_IDEM_AppleMesh02_AppleMesh04_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_4way_IDEM_AppleMesh03_AppleMesh08_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_6way_IDEM_AppleMesh02_AppleMesh03_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_6way_IDEM_AppleMesh02_AppleMesh04_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_6way_IDEM_AppleMesh03_AppleMesh08_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_8way_IDEM_AppleMesh02_AppleMesh03_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_8way_IDEM_AppleMesh02_AppleMesh04_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_8way_IDEM_AppleMesh03_AppleMesh08_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_10way_IDEM_AppleMesh02_AppleMesh03_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_10way_IDEM_AppleMesh02_AppleMesh04_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_10way_IDEM_AppleMesh03_AppleMesh08_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_12way_IDEM_AppleMesh02_AppleMesh03_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_12way_IDEM_AppleMesh02_AppleMesh04_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_12way_IDEM_AppleMesh03_AppleMesh08_colorbg_seed77_model_best',
]

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

filename_postfix = ''
save_label = False

for model_name in model_names_list:
    # flatten & subsample for previous layers of resnet50
    if 'layer' in model_name or "subsampled" in model_name:
        if "layerC2" in model_name and "layerC2_subsampled" not in model_name:
            subsample = False
        else:
            subsample = True
    else:
        subsample = False
    
    # load model
    model = load_model(model_name)
    model.eval()
    
    # add hook
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    if 'simplecnn' in model_name:
        model.fc2.register_forward_hook(get_activation('feats'))
    elif 'alexnet' in model_name:
        if 'layer4' in model_name:
            print("add hook in layer4")
            model.features[4].register_forward_hook(get_activation('feats'))
        elif 'layer8' in model_name:
            print("add hook in layer8")
            model.features[8].register_forward_hook(get_activation('feats'))
        elif 'layer12' in model_name:
            print("add hook in layer12")
            model.features[12].register_forward_hook(get_activation('feats'))
        elif 'layerC2' in model_name:
            model.classifier[2].register_forward_hook(get_activation('feats'))
        else: # final layer
            model.classifier[5].register_forward_hook(get_activation('feats'))
    elif model_name == 'vgg16':
        model.classifier[4].register_forward_hook(get_activation('feats'))
    elif model_name == 'vggface':
        model.fc7.register_forward_hook(get_activation('feats'))
    elif 'onlyDM' in model_name:
        model.depth_predictor.encoder[4].register_forward_hook(get_activation('feats'))
    elif 'head' in model_name :
        model.module.backbone.avgpool.register_forward_hook(get_activation('feats'))
    elif 'remapping' in model_name :
        model.module.backbone.avgpool.register_forward_hook(get_activation('feats'))
    else: # resnet50
        if 'layer1' in model_name:
            print("add hook in layer1")
            model.layer1.register_forward_hook(get_activation('feats'))
        elif 'layer2' in model_name:
            print("add hook in layer2")
            model.layer2.register_forward_hook(get_activation('feats'))
        elif 'layer3' in model_name:
            print("add hook in layer3")
            model.layer3.register_forward_hook(get_activation('feats'))
        elif 'layer4_no_pooling' in model_name:
            print("add hook in layer4_no_pooling")
            model.layer4.register_forward_hook(get_activation('feats'))
        else:
            model.avgpool.register_forward_hook(get_activation('feats'))
        
    # load data
    for exp_index, exp_name in enumerate(exp_names_list):
        print(model_name, exp_name)
        valdir = exp_name
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        _trans = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

        ## read & sort img filenames
        all_filenames = []
        # read img filenames
        for filename in os.listdir(valdir):
            if filename[-4:]=='.png':
                # for basal dataset / new controled exps
                assert(filename[:13]=='canvasvisible' or filename[:8]=='trialnum')
                all_filenames.append(filename)
            
                
        ## sort img filenames -> fix bug of sorting by value
        # for basal dataset / new controled exps
        if all_filenames[0][:13]=='canvasvisible':
            all_filenames = sorted(all_filenames, key=lambda x: int(x[:-4].split("_")[-1][5:]))
        elif all_filenames[0][:8]=='trialnum':
            all_filenames = sorted(all_filenames, key=lambda x: int(x[8:-4]))
        else:
            assert False, 'unsupported filename'
        print(len(all_filenames))
        
        # extract feats
        FEATS = []
        LABELS = []
        # loop through batches
        for idx, filename in enumerate(all_filenames):
            # read & transform img
            img = Image.open(os.path.join(valdir, filename)).convert("RGB")
            img_trans = _trans(img)
            
            # move to device
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            inputs = img_trans.unsqueeze(0).to(device)  # Move inputs to the same device as the model
            # forward pass [with feature extraction]
            preds = model(inputs)
            # preds = model.forward_single(inputs)
            # add feats and preds to lists
            # if 'simplecnn' in model_name:
            #     activation['feats'] = F.relu(activation['feats'])
            FEATS.append(activation['feats'].cpu().squeeze().unsqueeze(0).numpy())
            if save_label:
                LABELS.append(outputs.cpu().squeeze().unsqueeze(0).numpy())
        FEATS = np.concatenate(FEATS, axis=0)
        if save_label:
            LABELS = np.concatenate(LABELS, axis=0)
        print(FEATS.shape)
        
        if subsample:
            FEATS = FEATS.reshape([FEATS.shape[0], -1])
            if exp_index == 0: # use the same subset indexes for all exp
                # sampled_indexes = np.random.permutation()
                # sampled_indexes = np.random.permutation(FEATS.shape[1])[:2048]
                sampled_indexes = np.random.choice(FEATS.shape[1], 2048, replace=False)
            FEATS = FEATS[:, sampled_indexes]
            print(f"subsample to 2048 feats, {FEATS.shape}")
        
        # save feats
        filename = exp_name+'_'+model_name+filename_postfix+'.pth'
        torch.save(FEATS, filename)
        print(f'saving to {filename}\n')
        
        if save_label:
            filename = exp_name+'_'+model_name+filename_postfix+'_label.pth'
            torch.save(LABELS, filename)


# feats

In [ ]:
# Human data simulation (7-way emotion)
# exp_names_list_GER_AppleMesh00_fixedparams: multiway classficiation
# (1) exp_names_list_GER_AppleMesh00_fixedparams
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Ahana/data/Emotion_Identity_Stimuli_HiRes4000/Emotion/'] * 21
exp_name_list = [
                ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_disgust4_fixedparams'],
                 ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_fear4_fixedparams'],
                 ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_happiness4_fixedparams'],
                 ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_neutral_fixedparams'],
                 ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_sadness4_fixedparams'],
                 ['im20_AppleMesh00_anger4_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
                 ['im20_AppleMesh00_disgust4_fixedparams', 'im20_AppleMesh00_fear4_fixedparams'],
                 ['im20_AppleMesh00_disgust4_fixedparams', 'im20_AppleMesh00_happiness4_fixedparams'],
                 ['im20_AppleMesh00_disgust4_fixedparams', 'im20_AppleMesh00_neutral_fixedparams'],
                 ['im20_AppleMesh00_disgust4_fixedparams', 'im20_AppleMesh00_sadness4_fixedparams'],
                 ['im20_AppleMesh00_disgust4_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
                 ['im20_AppleMesh00_fear4_fixedparams', 'im20_AppleMesh00_happiness4_fixedparams'],
                 ['im20_AppleMesh00_fear4_fixedparams', 'im20_AppleMesh00_neutral_fixedparams'],
                 ['im20_AppleMesh00_fear4_fixedparams', 'im20_AppleMesh00_sadness4_fixedparams'],
                 ['im20_AppleMesh00_fear4_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
                 ['im20_AppleMesh00_happiness4_fixedparams', 'im20_AppleMesh00_neutral_fixedparams'],
                 ['im20_AppleMesh00_happiness4_fixedparams', 'im20_AppleMesh00_sadness4_fixedparams'],
                 ['im20_AppleMesh00_happiness4_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
                 ['im20_AppleMesh00_neutral_fixedparams', 'im20_AppleMesh00_sadness4_fixedparams'],
                 ['im20_AppleMesh00_neutral_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
                 ['im20_AppleMesh00_sadness4_fixedparams', 'im20_AppleMesh00_surprise4_fixedparams'],
]
emotion_names = ['anger4', 'disgust4', 'fear4', 'happiness4', 'neutral', 'sadness4', 'surprise4']
num_classes = 7
model_names_list = ['SL_colorbg_resnet50_finetune_7way_EM_AppleMesh02_seed77_model_best']

# specify which dataset to use by index 
for dataset_index in np.arange(1):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]
    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}
    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 
        # read feats
        inputs_all = []
        labels_all = []
        for class_idx, id_name in enumerate(emotion_names):
            os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
            path = os.path.join(data_root, f'im20_AppleMesh00_{id_name}_fixedparams_{model_name}{filename_postfix}.pth')
            print(f"loading {path}")
            features = torch.load(path)
            features = features.reshape(len(features), -1)
            inputs_all.append(features)
            labels_all.append(np.full(len(features), class_idx))
        inputs_all = np.concatenate(inputs_all, axis=0)
        labels_all = np.concatenate(labels_all, axis=0)
        dists = np.nan * np.ones((2, num_rep, len(labels_all))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(labels_all))) 
        # iter num_rep
        conf_mat = np.zeros((7,7))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []
            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(inputs_all, labels_all)):
                # split data
                X_train = inputs_all[train_index]
                X_test = inputs_all[test_index]
                y_train = labels_all[train_index]
                y_test = labels_all[test_index]
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).mean()
                _conf_mat = confusion_matrix(y_test, y_predict, labels=np.arange(num_classes))
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[split_index, rep_index, test_index] = _acc_per_img
            all_scores.append(np.array(scores).mean())
        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc
        #get task name
        name1 = exp_name_0.split('_')
        name2 = exp_name_1.split('_')
        newname = name1[2] + name2[2]
        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name][newname] = {}
        useful_stats[data_root][model_name][newname]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name][newname]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name][newname]['dist'] = dists

In [ ]:
print("Label distribution:", np.bincount(labels_all))
per_class_acc = np.diag(conf_mat) / conf_mat.sum(axis=1)
print("Per-class accuracy:", per_class_acc)
print("Mean per-class accuracy:", per_class_acc.mean())


# extract pixels

In [ ]:
# extract pixels

for exp_name in exp_names_list:
    valdir = exp_name

    ## read & sort img filenames
    all_filenames = []
    # read img filenames
    for filename in os.listdir(valdir):
        if filename[-4:]=='.png':
            # for basal dataset / new controled exps
            # assert(filename[:13]=='canvasvisible' or filename[:8]=='trialnum')
            all_filenames.append(filename)

    ## sort img filenames -> fix bug of sorting by value
    # for basal dataset / new controled exps
    if all_filenames[0][:13]=='canvasvisible':
        all_filenames = sorted(all_filenames, key=lambda x: int(x[:-4].split("_")[-1][5:]))
    elif all_filenames[0][:8]=='trialnum':
        all_filenames = sorted(all_filenames, key=lambda x: int(x[8:-4]))
    # else:
    #     assert False, 'unsupported filename'
    print(len(all_filenames))
    
    # extract feats
    FEATS = []
    # loop through batches
    for idx, filename in enumerate(all_filenames):
        # read & transform img
        img = Image.open(os.path.join(valdir, filename)).convert('L')
        img = img.resize((64, 64))
        FEATS.append(np.array(img).reshape(-1))
    FEATS = np.array(FEATS)
    print(FEATS.shape)

    # save feats
    filename = exp_name+'_'+'pixel'+'.pth'
    torch.save(FEATS, filename)
    print(f'saving to {filename}\n')

# get bio i1s - GFR (AppleMesh00 vs AppleMesh01)

In [ ]:
# read file that contains biological i1 data

# Human i1, GFR (AppleMesh00-AppleMesh01)
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_AppleMesh01/human_n_subs=37_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=35245_n_rep=5"
_path = "../data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json_n_rep=5"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json_n_rep=5"
with open(_path, 'rb') as f:
    human_i1_5rep = pickle.load(f).squeeze()

# Monkey i1, GFR (AppleMesh00-AppleMesh01)
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_AppleMesh01/AJ_176_20211011_Var6vbsl_set0_im151_AppleMesh01_dur400ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh00_dur400ms_lab_updated.json_0_I1_partition_splithalf_trial_binsize=24008n_rep=5"
_path = "../data/face_data/bio_data/avg-monkey-i1_splithalf"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/bio_data/avg-monkey-i1_splithalf"
with open(_path, 'rb') as f:
    marmoset_i1_5rep = pickle.load(f).squeeze()

# Human i1, GFR (AppleMesh00-AppleMesh01) - no rep
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_AppleMesh01/human_n_subs=37_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json_I1_partition_trial_binsize=35245"
_path = "../data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_AppleMesh00_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh01_dur200ms_lab_updated.json"
with open(_path, 'rb') as f:
    human_i1 = pickle.load(f).squeeze().reshape(-1)

# Monkey i1, GFR (AppleMesh00-AppleMesh01) - no rep
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_AppleMesh01/AJ_176_20211011_Var6vbsl_set0_im151_AppleMesh01_dur400ms_lab_updated.json_20211011_Var6vbsl_set0_im151_AppleMesh00_dur400ms_lab_updated.json_0_I1_partition_trial_binsize=24008"
_path = "../data/face_data/bio_data/avg-monkey-i1"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/bio_data/avg-monkey-i1"
with open(_path, 'rb') as f:
    marmoset_i1 = pickle.load(f).squeeze().reshape(-1)

In [ ]:
# normalize 
def NormalizeData(data, data_max=None, data_min=None): # normalize to [0,1]
    if data_max is None and data_min is None:
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    else:
        return (data - data_min) / (data_max - data_min)
    
is_normalized = True

if is_normalized:
    human_i1_5rep = NormalizeData(human_i1_5rep, 4, -4)
    human_i1 = NormalizeData(human_i1, 4, -4)
    marmoset_i1_5rep = NormalizeData(marmoset_i1_5rep, 4, -4)
    marmoset_i1 = NormalizeData(marmoset_i1, 4, -4)
human_i1_5rep.shape, marmoset_i1_5rep.shape, human_i1.shape, marmoset_i1.shape

In [ ]:
human_i1_5rep.shape, marmoset_i1_5rep.shape, human_i1.shape, marmoset_i1.shape

# get human i1s - GOR (camel vs elephant)

In [ ]:
# read human i1s - camel vs. elephant

# Human experiment, using set0, im151 -- elephant, subs=31, 20211011
_path = "../../data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant/human_n_subs=31_20211011_Var6vbsl_set0_im151_camel_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elephant_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=13738_n_rep=5"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant/human_n_subs=31_20211011_Var6vbsl_set0_im151_camel_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elephant_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=13738_n_rep=5"
with open(_path, 'rb') as f:
    human_i1s_obj_vbsl151 = pickle.load(f).squeeze()

# Human experiment, using set1, im101 -- elephant, subs=26, 20220412 
_path = "../../data/face_data/control_exp/objs/20220412_Var6vb_same_as_camel_sp_set1_im101_elephant/human_n_subs=26_20220412_Var6vbsp_set1_im101_camel_dur200ms.json_20220412_Var6vb_same_as_camel_sp_set1_im101_elephant_dur200ms.json_I1_partition_splithalf_trial_binsize=10043_n_rep=5"
_path = "/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/20220412_Var6vb_same_as_camel_sp_set1_im101_elephant/human_n_subs=26_20220412_Var6vbsp_set1_im101_camel_dur200ms.json_20220412_Var6vb_same_as_camel_sp_set1_im101_elephant_dur200ms.json_I1_partition_splithalf_trial_binsize=10043_n_rep=5"  
with open(_path, 'rb') as f:
    human_i1s_obj_vbsp101 = pickle.load(f).squeeze()

In [ ]:
is_normalized = True
if is_normalized:
    human_i1s_obj_vbsl151 = NormalizeData(human_i1s_obj_vbsl151)
    human_i1s_obj_vbsp101 = NormalizeData(human_i1s_obj_vbsp101)

In [ ]:
human_i1s_obj_vbsl151.shape, human_i1s_obj_vbsp101.shape

# compute model i1s

In [ ]:
# (1) exp_names_list_basics

useful_stats = {}

data_root_list = [
                '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/',
                '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
                '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_AppleMesh03_praneeth/',
                '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood/',
                '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/',
                ]

exp_name_list = [
                ['20211011_Var6vbsl_set0_im151_AppleMesh00', '20211011_Var6vbsl_set0_im151_AppleMesh01'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral'],
                ['20230406_Var6vbsl_set0_im151_praneeth_dur200ms_lab', '20230406_Var6vbsl_set0_im151_AppleMesh03_dur200ms_lab'],
                ['20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_dur200ms_lab', '20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_dur200ms_lab'],
                ['20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_notexture_dur200ms_lab', '20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_notexture_dur200ms_lab'],
                ]

for dataset_index in [0,1,2,3,4]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)
        
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (2) exp_names_list_GFR

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/'] * 66
exp_name_list = [
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral'],
                 ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral'],
                 ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral'],
                 ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral'],
                 ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral'],
                 ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                 ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral'],
                 ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral'],
                 ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral'],
                 ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral'],
                 ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                 ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral'],
                 ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral'],
                 ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral'],
                 ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                 ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral'],
                 ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral'],
                 ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                 ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral'],
                 ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                 ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh00/vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh01/vbsle151_AppleMesh01_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh02/vbsle151_AppleMesh02_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh03/vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh04/vbsle151_AppleMesh04_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh05/vbsle151_AppleMesh05_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral', 'vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral'],
                ['vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh06/vbsle151_AppleMesh06_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral', 'vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral'],
                ['vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh07/vbsle151_AppleMesh07_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral', 'vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral'],
                ['vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh08/vbsle151_AppleMesh08_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral', 'vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral'],
                ['vbsle151_AppleMesh09/vbsle151_AppleMesh09_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
                ['vbsle151_AppleMesh10/vbsle151_AppleMesh10_neutral', 'vbsle151_AppleMesh11/vbsle151_AppleMesh11_neutral'],
]
    
# specify which dataset to use by index 
for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (3) exp_names_list_CTFR

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'] * 66
exp_name_list = [
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh01_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh02_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh02_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh06_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh09_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh09_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh10_neutral', 'vbsl151_AppleMesh11_neutral'],
]

    
# specify which dataset to use by index 
for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (4) exp_names_list_GTFR

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/'] * 66
exp_name_list = [
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh01_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh02_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh02_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh03_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh04_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh05_neutral'],
                 ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh06_neutral'],
                 ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh06_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh01_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh02_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh03_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh04_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh05_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh07_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh06_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh08_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh07_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh09_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh08_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh09_neutral', 'vbsl151_AppleMesh10_neutral'],
                ['vbsl151_AppleMesh09_neutral', 'vbsl151_AppleMesh11_neutral'],
                ['vbsl151_AppleMesh10_neutral', 'vbsl151_AppleMesh11_neutral'],
]
    
# specify which dataset to use by index 
for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (5) exp_names_list_GER_AppleMesh03
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh03/'] * 21
exp_name_list = [
                ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_happiness_4'],
                 ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_sadness_4'],
                 ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_disgust_4'],
                 ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_fear_4'],
                 ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_anger_4'],
                 ['vbsle151_AppleMesh03_neutral', 'vbsle151_AppleMesh03_surprise_4'],
                 ['vbsle151_AppleMesh03_happiness_4', 'vbsle151_AppleMesh03_sadness_4'],
                 ['vbsle151_AppleMesh03_happiness_4', 'vbsle151_AppleMesh03_disgust_4'],
                 ['vbsle151_AppleMesh03_happiness_4', 'vbsle151_AppleMesh03_fear_4'],
                 ['vbsle151_AppleMesh03_happiness_4', 'vbsle151_AppleMesh03_anger_4'],
                 ['vbsle151_AppleMesh03_happiness_4', 'vbsle151_AppleMesh03_surprise_4'],
                 ['vbsle151_AppleMesh03_sadness_4', 'vbsle151_AppleMesh03_disgust_4'],
                 ['vbsle151_AppleMesh03_sadness_4', 'vbsle151_AppleMesh03_fear_4'],
                 ['vbsle151_AppleMesh03_sadness_4', 'vbsle151_AppleMesh03_anger_4'],
                 ['vbsle151_AppleMesh03_sadness_4', 'vbsle151_AppleMesh03_surprise_4'],
                 ['vbsle151_AppleMesh03_disgust_4', 'vbsle151_AppleMesh03_fear_4'],
                 ['vbsle151_AppleMesh03_disgust_4', 'vbsle151_AppleMesh03_anger_4'],
                 ['vbsle151_AppleMesh03_disgust_4', 'vbsle151_AppleMesh03_surprise_4'],
                 ['vbsle151_AppleMesh03_fear_4', 'vbsle151_AppleMesh03_anger_4'],
                 ['vbsle151_AppleMesh03_fear_4', 'vbsle151_AppleMesh03_surprise_4'],
                 ['vbsle151_AppleMesh03_anger_4', 'vbsle151_AppleMesh03_surprise_4']

                ]


# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:

    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists


In [ ]:
# (6) exp_names_list_GER_AppleMesh00
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_AppleMesh00/'] * 21
exp_name_list = [
                ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_happiness_4'],
                 ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_sadness_4'],
                 ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_disgust_4'],
                 ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_fear_4'],
                 ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_anger_4'],
                 ['vbsle151_AppleMesh00_neutral', 'vbsle151_AppleMesh00_surprise_4'],
                 ['vbsle151_AppleMesh00_happiness_4', 'vbsle151_AppleMesh00_sadness_4'],
                 ['vbsle151_AppleMesh00_happiness_4', 'vbsle151_AppleMesh00_disgust_4'],
                 ['vbsle151_AppleMesh00_happiness_4', 'vbsle151_AppleMesh00_fear_4'],
                 ['vbsle151_AppleMesh00_happiness_4', 'vbsle151_AppleMesh00_anger_4'],
                 ['vbsle151_AppleMesh00_happiness_4', 'vbsle151_AppleMesh00_surprise_4'],
                 ['vbsle151_AppleMesh00_sadness_4', 'vbsle151_AppleMesh00_disgust_4'],
                 ['vbsle151_AppleMesh00_sadness_4', 'vbsle151_AppleMesh00_fear_4'],
                 ['vbsle151_AppleMesh00_sadness_4', 'vbsle151_AppleMesh00_anger_4'],
                 ['vbsle151_AppleMesh00_sadness_4', 'vbsle151_AppleMesh00_surprise_4'],
                 ['vbsle151_AppleMesh00_disgust_4', 'vbsle151_AppleMesh00_fear_4'],
                 ['vbsle151_AppleMesh00_disgust_4', 'vbsle151_AppleMesh00_anger_4'],
                 ['vbsle151_AppleMesh00_disgust_4', 'vbsle151_AppleMesh00_surprise_4'],
                 ['vbsle151_AppleMesh00_fear_4', 'vbsle151_AppleMesh00_anger_4'],
                 ['vbsle151_AppleMesh00_fear_4', 'vbsle151_AppleMesh00_surprise_4'],
                 ['vbsle151_AppleMesh00_anger_4', 'vbsle151_AppleMesh00_surprise_4']
                ]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10, 11, 12, 13,14, 15, 16, 17, 18, 19, 20]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (7) exp_names_list_CTER_AppleMesh00
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'] * 21
exp_name_list = [
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_happiness_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_sadness_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_sadness_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_fear_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_fear_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_anger_4', 'vbsl151_AppleMesh00_surprise_4']
]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (8) exp_names_list_GTER_AppleMesh00
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/'] * 21
exp_name_list = [
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_happiness_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_sadness_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_neutral', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_sadness_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_happiness_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_disgust_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_sadness_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_fear_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_disgust_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_fear_4', 'vbsl151_AppleMesh00_anger_4'],
                ['vbsl151_AppleMesh00_fear_4', 'vbsl151_AppleMesh00_surprise_4'],
                ['vbsl151_AppleMesh00_anger_4', 'vbsl151_AppleMesh00_surprise_4']
]

# specify which dataset to use by index 
for dataset_index in [0, 1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19,20]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (9) exp_names_list_basel_GFR

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/'] * 66
exp_name_list = [
                ['id10', 'id11'],
                 ['id10', 'id12'],
                 ['id10', 'id13'],
                 ['id10', 'id14'],
                 ['id10', 'id15'],
                 ['id10', 'id16'],
                 ['id11', 'id12'],
                 ['id11', 'id13'],
                 ['id11', 'id14'],
                 ['id11', 'id15'],
                 ['id11', 'id16'],
                 ['id12', 'id13'],
                 ['id12', 'id14'],
                 ['id12', 'id15'],
                 ['id12', 'id16'],
                 ['id13', 'id14'],
                 ['id13', 'id15'],
                 ['id13', 'id16'],
                 ['id14', 'id15'],
                 ['id14', 'id16'],
                 ['id15', 'id16'],
                ['id10', 'id17'],
                ['id10', 'id18'],
                ['id10', 'id19'],
                ['id10', 'id20'],
                ['id10', 'id21'],
                ['id11', 'id17'],
                ['id11', 'id18'],
                ['id11', 'id19'],
                ['id11', 'id20'],
                ['id11', 'id21'],
                ['id12', 'id17'],
                ['id12', 'id18'],
                ['id12', 'id19'],
                ['id12', 'id20'],
                ['id12', 'id21'],
                ['id13', 'id17'],
                ['id13', 'id18'],
                ['id13', 'id19'],
                ['id13', 'id20'],
                ['id13', 'id21'],
                ['id14', 'id17'],
                ['id14', 'id18'],
                ['id14', 'id19'],
                ['id14', 'id20'],
                ['id14', 'id21'],
                ['id15', 'id17'],
                ['id15', 'id18'],
                ['id15', 'id19'],
                ['id15', 'id20'],
                ['id15', 'id21'],
                ['id16', 'id17'],
                ['id16', 'id18'],
                ['id16', 'id19'],
                ['id16', 'id20'],
                ['id16', 'id21'],
                ['id17', 'id18'],
                ['id17', 'id19'],
                ['id17', 'id20'],
                ['id17', 'id21'],
                ['id18', 'id19'],
                ['id18', 'id20'],
                ['id18', 'id21'],
                ['id19', 'id20'],
                ['id19', 'id21'],
                ['id20', 'id21'],
] 
   
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}

# specify which dataset to use by index 
for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

# Print model accuracies across tasks 
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))

In [ ]:
# (10) exp_names_list_basel_CTFR

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/'] * 66
exp_name_list = [
                 ['id10', 'id11'],
                 ['id10', 'id12'],
                 ['id10', 'id13'],
                 ['id10', 'id14'],
                 ['id10', 'id15'],
                 ['id10', 'id16'],
                 ['id11', 'id12'],
                 ['id11', 'id13'],
                 ['id11', 'id14'],
                 ['id11', 'id15'],
                 ['id11', 'id16'],
                 ['id12', 'id13'],
                 ['id12', 'id14'],
                 ['id12', 'id15'],
                 ['id12', 'id16'],
                 ['id13', 'id14'],
                 ['id13', 'id15'],
                 ['id13', 'id16'],
                 ['id14', 'id15'],
                 ['id14', 'id16'],
                 ['id15', 'id16'],
                ['id10', 'id17'],
                ['id10', 'id18'],
                ['id10', 'id19'],
                ['id10', 'id20'],
                ['id10', 'id21'],
                ['id11', 'id17'],
                ['id11', 'id18'],
                ['id11', 'id19'],
                ['id11', 'id20'],
                ['id11', 'id21'],
                ['id12', 'id17'],
                ['id12', 'id18'],
                ['id12', 'id19'],
                ['id12', 'id20'],
                ['id12', 'id21'],
                ['id13', 'id17'],
                ['id13', 'id18'],
                ['id13', 'id19'],
                ['id13', 'id20'],
                ['id13', 'id21'],
                ['id14', 'id17'],
                ['id14', 'id18'],
                ['id14', 'id19'],
                ['id14', 'id20'],
                ['id14', 'id21'],
                ['id15', 'id17'],
                ['id15', 'id18'],
                ['id15', 'id19'],
                ['id15', 'id20'],
                ['id15', 'id21'],
                ['id16', 'id17'],
                ['id16', 'id18'],
                ['id16', 'id19'],
                ['id16', 'id20'],
                ['id16', 'id21'],
                ['id17', 'id18'],
                ['id17', 'id19'],
                ['id17', 'id20'],
                ['id17', 'id21'],
                ['id18', 'id19'],
                ['id18', 'id20'],
                ['id18', 'id21'],
                ['id19', 'id20'],
                ['id19', 'id21'],
                ['id20', 'id21'],
] 
   
    
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}

for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

# Print model accuracies across tasks 
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))


In [ ]:
# (11) exp_names_list_basel_GFR_ver2 (id988-999)
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/'] * 66
exp_name_list = [
                ['id988', 'id989'],
                 ['id988', 'id990'],
                 ['id988', 'id991'],
                 ['id988', 'id992'],
                 ['id988', 'id993'],
                 ['id988', 'id994'],
                 ['id989', 'id990'],
                 ['id989', 'id991'],
                 ['id989', 'id992'],
                 ['id989', 'id993'],
                 ['id989', 'id994'],
                 ['id990', 'id991'],
                 ['id990', 'id992'],
                 ['id990', 'id993'],
                 ['id990', 'id994'],
                 ['id991', 'id992'],
                 ['id991', 'id993'],
                 ['id991', 'id994'],
                 ['id992', 'id993'],
                 ['id992', 'id994'],
                 ['id993', 'id994'],
                ['id988', 'id995'],
                ['id988', 'id996'],
                ['id988', 'id997'],
                ['id988', 'id998'],
                ['id988', 'id999'],
                ['id989', 'id995'],
                ['id989', 'id996'],
                ['id989', 'id997'],
                ['id989', 'id998'],
                ['id989', 'id999'],
                ['id990', 'id995'],
                ['id990', 'id996'],
                ['id990', 'id997'],
                ['id990', 'id998'],
                ['id990', 'id999'],
                ['id991', 'id995'],
                ['id991', 'id996'],
                ['id991', 'id997'],
                ['id991', 'id998'],
                ['id991', 'id999'],
                ['id992', 'id995'],
                ['id992', 'id996'],
                ['id992', 'id997'],
                ['id992', 'id998'],
                ['id992', 'id999'],
                ['id993', 'id995'],
                ['id993', 'id996'],
                ['id993', 'id997'],
                ['id993', 'id998'],
                ['id993', 'id999'],
                ['id994', 'id995'],
                ['id994', 'id996'],
                ['id994', 'id997'],
                ['id994', 'id998'],
                ['id994', 'id999'],
                ['id995', 'id996'],
                ['id995', 'id997'],
                ['id995', 'id998'],
                ['id995', 'id999'],
                ['id996', 'id997'],
                ['id996', 'id998'],
                ['id996', 'id999'],
                ['id997', 'id998'],
                ['id997', 'id999'],
                ['id998', 'id999'],
] 
   
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}

# specify which dataset to use by index 
for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

# Print model accuracies across tasks 
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))

In [ ]:
# (12) exp_names_list_basel_CTFR_ver2 (id988-999)
useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/'] * 66
exp_name_list = [
                ['id988', 'id989'],
                 ['id988', 'id990'],
                 ['id988', 'id991'],
                 ['id988', 'id992'],
                 ['id988', 'id993'],
                 ['id988', 'id994'],
                 ['id989', 'id990'],
                 ['id989', 'id991'],
                 ['id989', 'id992'],
                 ['id989', 'id993'],
                 ['id989', 'id994'],
                 ['id990', 'id991'],
                 ['id990', 'id992'],
                 ['id990', 'id993'],
                 ['id990', 'id994'],
                 ['id991', 'id992'],
                 ['id991', 'id993'],
                 ['id991', 'id994'],
                 ['id992', 'id993'],
                 ['id992', 'id994'],
                 ['id993', 'id994'],
                ['id988', 'id995'],
                ['id988', 'id996'],
                ['id988', 'id997'],
                ['id988', 'id998'],
                ['id988', 'id999'],
                ['id989', 'id995'],
                ['id989', 'id996'],
                ['id989', 'id997'],
                ['id989', 'id998'],
                ['id989', 'id999'],
                ['id990', 'id995'],
                ['id990', 'id996'],
                ['id990', 'id997'],
                ['id990', 'id998'],
                ['id990', 'id999'],
                ['id991', 'id995'],
                ['id991', 'id996'],
                ['id991', 'id997'],
                ['id991', 'id998'],
                ['id991', 'id999'],
                ['id992', 'id995'],
                ['id992', 'id996'],
                ['id992', 'id997'],
                ['id992', 'id998'],
                ['id992', 'id999'],
                ['id993', 'id995'],
                ['id993', 'id996'],
                ['id993', 'id997'],
                ['id993', 'id998'],
                ['id993', 'id999'],
                ['id994', 'id995'],
                ['id994', 'id996'],
                ['id994', 'id997'],
                ['id994', 'id998'],
                ['id994', 'id999'],
                ['id995', 'id996'],
                ['id995', 'id997'],
                ['id995', 'id998'],
                ['id995', 'id999'],
                ['id996', 'id997'],
                ['id996', 'id998'],
                ['id996', 'id999'],
                ['id997', 'id998'],
                ['id997', 'id999'],
                ['id998', 'id999'],
] 
   
    
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}

for dataset_index in range(66):
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_AppleMesh00, imageidx_AppleMesh01)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

# Print model accuracies across tasks 
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))


In [ ]:
# (13) exp_names_list_lfw
useful_stats = {}
data_root_list = [    
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell_inverted/',
]

exp_name_list = [['Bush', 'Powell'],
['Bush', 'Powell']]

for dataset_index in [0,1]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (14) exp_names_list_ytface
useful_stats = {}

data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/'
]

exp_name_list = [['Stanca', 'Beard'],
['Kennedy', 'Macdonald']]

for dataset_index in [0,1]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (15) exp_names_list_GOR
useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dog_horse/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_gray_downsampled/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_downsampled/'
]
exp_name_list = [
                ['20211011_Var6vbsl_set0_im151_camel', '20211011_Var6vbsl_set0_im151_elephant'],
                ['vbsle_151_dog', 'vbsle_151_horse'],
                ['camel_colored_gray', 'elephant_colored_gray'],
                ['camel_colored', 'elephant_colored']
                 ]

for dataset_index in [0,1,2,3]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

In [ ]:
# (16) exp_names_list_imagenet
useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/shark151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/bird151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/vehicle151'
]
exp_name_list = [
                 ['shark1', 'shark2'],
                 ['bird1', 'bird2'],
                 ['bike', 'car']
                 ]

for dataset_index in [0,1,2]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)

        # cross-validation
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

# Print in-distribution and out-of-distribution acc

In [ ]:
def compute_ood_iid_acc(pairs, acc_values, ood_set):
    """
    - pairs: list of 66 name pairs (e.g., ["AppleMesh00_AppleMesh01", "AppleMesh00_AppleMesh02E", ...])
    - acc_values: list of 66 accuracy values
    - ood_set: set of OOD IDs (e.g., {"AppleMesh00", "AppleMesh01"} for 2-ID OOD).
    
    Returns:
    - overall_acc: all mean
    - overall_std: all std
    - ood_acc: OOD mean (for pairs not containing "OOD IDs").
    - ood_std: OOD std
    - iid_acc: IID mean
    - iid_std: IID std
    """
    ood_indices = []  # Pairs that DO NOT contain any OOD IDs
    iid_indices = []  # Pairs that CONTAIN at least one OOD ID

    for i, pair in enumerate(pairs):
        name1, name2 = pair.split("_")
        # If neither name1 nor name2 is in OOD set, classify as OOD
        if name1 not in ood_set and name2 not in ood_set:
            ood_indices.append(i)
        else:
            iid_indices.append(i)

    # Compute mean accuracies
    overall_acc = np.mean(acc_values)
    overall_std = np.std(acc_values)

    ood_acc = np.mean([acc_values[i] for i in ood_indices]) if ood_indices else None
    ood_std = np.std([acc_values[i] for i in ood_indices]) if ood_indices else None

    iid_acc = np.mean([acc_values[i] for i in iid_indices]) if iid_indices else None
    iid_std = np.std([acc_values[i] for i in iid_indices]) if iid_indices else None
    print([acc_values[i] for i in ood_indices])
    return overall_acc, overall_std, ood_acc, ood_std, iid_acc, iid_std, len(ood_indices), len(iid_indices)

# Example usage
pairs = [
    "AppleMesh00_AppleMesh01", "AppleMesh00_AppleMesh02", "AppleMesh00_AppleMesh03", "AppleMesh00_AppleMesh04", "AppleMesh00_AppleMesh05", "AppleMesh00_AppleMesh06",
    "AppleMesh01_AppleMesh02", "AppleMesh01_AppleMesh03", "AppleMesh01_AppleMesh04", "AppleMesh01_AppleMesh05", "AppleMesh01_AppleMesh06",
    "AppleMesh02_AppleMesh03", "AppleMesh02_AppleMesh04", "AppleMesh02_AppleMesh05", "AppleMesh02_AppleMesh06", "AppleMesh03_AppleMesh04",
    "AppleMesh03_AppleMesh05", "AppleMesh03_AppleMesh06", "AppleMesh04_AppleMesh05", "AppleMesh04_AppleMesh06", "AppleMesh05_AppleMesh06",
    "AppleMesh00_AppleMesh07", "AppleMesh00_AppleMesh08", "AppleMesh00_AppleMesh09", "AppleMesh00_AppleMesh10", "AppleMesh00_AppleMesh11",
    "AppleMesh01_AppleMesh07", "AppleMesh01_AppleMesh08", "AppleMesh01_AppleMesh09", "AppleMesh01_AppleMesh10", "AppleMesh01_AppleMesh11",
    "AppleMesh02_AppleMesh07", "AppleMesh02_AppleMesh08", "AppleMesh02_AppleMesh09", "AppleMesh02_AppleMesh10", "AppleMesh02_AppleMesh11",
    "AppleMesh03_AppleMesh07", "AppleMesh03_AppleMesh08", "AppleMesh03_AppleMesh09", "AppleMesh03_AppleMesh10", "AppleMesh03_AppleMesh11",
    "AppleMesh04_AppleMesh07", "AppleMesh04_AppleMesh08", "AppleMesh04_AppleMesh09", "AppleMesh04_AppleMesh10", "AppleMesh04_AppleMesh11",
    "AppleMesh05_AppleMesh07", "AppleMesh05_AppleMesh08", "AppleMesh05_AppleMesh09", "AppleMesh05_AppleMesh10", "AppleMesh05_AppleMesh11",
    "AppleMesh06_AppleMesh07", "AppleMesh06_AppleMesh08", "AppleMesh06_AppleMesh09", "AppleMesh06_AppleMesh10", "AppleMesh06_AppleMesh11",
    "AppleMesh07_AppleMesh08", "AppleMesh07_AppleMesh09", "AppleMesh07_AppleMesh10", "AppleMesh07_AppleMesh11",
    "AppleMesh08_AppleMesh09", "AppleMesh08_AppleMesh10", "AppleMesh08_AppleMesh11", "AppleMesh09_AppleMesh10", "AppleMesh09_AppleMesh11", "AppleMesh10_AppleMesh11"
]

acc_values = [
0.893467,	0.936926,	0.957427,	0.882339,	0.925035,	0.981286,	0.905658,	0.922682,	0.902667,	0.926330,	0.987154,	0.824061,	0.930411,	0.936452,	0.976455,	0.947849,	0.935982,	0.982017,	0.874466,	0.982350,	0.949014,	0.980628,	0.854192,	0.940664,	0.926195,	0.963151,	0.932549,	0.975100,	0.921263,	0.911155,	0.953658,	0.934628,	0.988645,	0.812758,	0.916327,	0.950502,	0.946234,	0.981722,	0.846096,	0.921899,	0.947048,	0.942186,	0.977653,	0.910598,	0.849870,	0.952589,	0.932289,	0.962734,	0.907123,	0.881646,	0.903197,	0.956275,	0.880066,	0.968479,	0.978896,	0.979047,	0.951364,	0.932040,	0.940130,	0.964267,	0.980344,	0.980041,	0.972848,	0.884662,	0.933679,	0.938574,
]

ood_set = {"AppleMesh02","AppleMesh03", "AppleMesh04", "AppleMesh08", "AppleMesh11", "AppleMesh06", "AppleMesh05", "AppleMesh10"}

# Compute and print results
overall_acc, overall_std, ood_acc, ood_std, iid_acc, iid_std, ood_len, iid_len = compute_ood_iid_acc(pairs, acc_values, ood_set)
print((iid_len), (ood_len))
print(f"Overall Acc: {overall_acc:.8f}, Overall Std: {overall_std:.8f}")
print(f"IID Acc: {iid_acc:.8f}, IID Std: {iid_std:.8f}")
print(f"OOD Acc: {ood_acc:.8f}, OOD Std: {ood_std:.8f}")
print(f"{overall_acc:.8f}	{overall_std:.8f}	{iid_acc:.8f}	{iid_std:.8f}	{ood_acc:.8f}	{ood_std:.8f}")

# Compute i1 and i1_5rep

In [ ]:
# useful_stats = {}
import pickle
with open('useful_stats_penn.pkl', 'rb') as f:
    useful_stats = pickle.load(f)
    
# hierachy: [task_dir][model_name][acc/acc_per_img/dist/i1]
# acc [list]: acc over all datapoints and trails -> [acc_mean, acc_std]
# acc_per_img [array]: acc per image averaged over all trails -> shape [2, num_img_per_class]
# dist [array]: all svm distance (note that the second class dist is negated) -> shape [2, num_repetition, num_img_per_class]
# i1_5rep [array]: random split-half i1 with 5 repetition -> shape [5, 2*num_img_per_class]
# i1 [array]: model i1 (the same as dist averaging over trails) -> shape [2, num_img_per_class]
# i1_corr [list]: model i1 correlation with bio systems. For vbsl101, the order is [Bourgeois, Sausage, AJ]. 
# For vbsl151, the order is [Human, AJ]. For each corr, I record [i1_corr_mean, i1_corr_std]

In [ ]:
# compute model i1 and i1_5rep
num_rep = 100
for data_root in useful_stats.keys():
    for model_name in useful_stats[data_root].keys():
        # pull SVM distances, shape (2, 100, N)
        i1_dists = useful_stats[data_root][model_name]['dist']
        print(i1_dists.size)
        avg_i1s = i1_dists.mean(1) # shape (2, N)
        i1_dists = np.concatenate((i1_dists[0], i1_dists[1]), axis=1)
        model_i1s = []
        # for each of 5 repeats, shuffle the 100 rows, average first and last 50 rows and concatenate
        for rep_index in range(5):
            np.random.shuffle(i1_dists)
            tmp = np.concatenate([i1_dists[:int(num_rep/2)].mean(0), 
                                  i1_dists[int(num_rep/2):].mean(0)])
            model_i1s.append(np.expand_dims(tmp, axis=0))
        model_i1s = np.concatenate(model_i1s, axis=0)
        
        useful_stats[data_root][model_name]['i1_5rep'] = model_i1s
        useful_stats[data_root][model_name]['i1'] = avg_i1s

        # normalize
        useful_stats[data_root][model_name]['i1_5rep'] = NormalizeData(model_i1s)
        useful_stats[data_root][model_name]['i1'] = NormalizeData(avg_i1s)


# print acc

In [ ]:
# print overall and subset accuracies

subset_list = [[0,2], [1,3], [4,5]]
subset_name_list = ['normal_face', 'inverted_face', 'reverse_contrast']
is_print_subset = True

for data_root in useful_stats.keys():
    print(data_root)
    if data_root != '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_101/':

        for model_name in useful_stats[data_root].keys():
            # print acc
            acc = useful_stats[data_root][model_name]['acc']
            print(f"{model_name}: %.4f±%.4f" % (acc[0]*100, acc[1]*100))
            # print subset acc
            tmp = useful_stats[data_root][model_name]['acc_per_img'].mean(0)
            if is_print_subset:
                tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
                for subset_id, subset in enumerate(subset_list):
                    # get acc
                    print("%s acc: %.4f" % (subset_name_list[subset_id], tmp_group[subset].mean()*100))
                    useful_stats[data_root][model_name][f'{subset_name_list[subset_id]}'] = tmp_group[subset].mean()
                print()
        print()

# compare model and bio i1s

In [ ]:
def subset_upright_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 0:25], array[:, 50:75], 
                            array[:, 151:151+25], array[:, 151+50:151+75], 
                            array[:, 302:302+25], array[:, 302+50:302+75], 
                            array[:, 302+151:302+151+25], array[:, 302+151+50:302+151+75], ], axis=1) 

# -------------- SL -----------------
def subset_inverted_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 25:50], array[:, 75:100], 
                            array[:, 151+25:151+50], array[:, 151+75:151+100], 
                            array[:, 302+25:302+50], array[:, 302+75:302+100], 
                            array[:, 302+151+25:302+151+50], array[:, 302+151+75:302+151+100], ], axis=1) 

# -------------- SL -----------------
def subset_reverse_contrast_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 100:150],  
                            array[:, 151+100:151+150],  
                            array[:, 302+100:302+150], 
                            array[:, 302+151+100:302+151+150], ], axis=1) 

In [ ]:
## compute noise-corrected i1 corr between model and bio i1s

upright_only = False  # set true to compute for only upright images of vbsli*

# select task 
bio_i1s_list = [human_i1_5rep, marmoset_i1_5rep] # GFR
# bio_i1s_list = [human_i1_5rep]
# bio_i1s_list = [human_i1s_vbsl]
# bio_i1s_list = [human_i1s_obj_vbsl151] # GOR

# select data root
data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/' # GFR
# data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/' # GOR


for model_name in model_names_list:
    # get model i1s
    model_i1s = useful_stats[data_root][model_name]['i1_5rep']
    if upright_only:
        model_i1s = subset_upright_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_upright'] = np.zeros((len(bio_i1s_list), 2))
    else:
        useful_stats[data_root][model_name]['i1_corr'] = np.zeros((len(bio_i1s_list), 2))
    for bio_index, bio_i1s in enumerate(bio_i1s_list):
        # get bio i1s
        bio_i1s = bio_i1s.squeeze()
        #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
        bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
        if upright_only:
            bio_i1s = subset_upright_5rep_604(bio_i1s)
        i1_corr = []
        half_length = int(model_i1s.shape[1]/2)
        bio_internal = []
        for rep_index in range(5):
            corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
            corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
            corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
            corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
            print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
            i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
            bio_internal.append(corr_4)
        if upright_only:
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][1] = np.std(np.array(i1_corr))
        else:
            useful_stats[data_root][model_name]['i1_corr'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr'][bio_index][1] = np.std(np.array(i1_corr))

    # ------------------------------- SL -------------------------------
    model_i1s_concat = np.concatenate((useful_stats[data_root][model_name]['i1'][0], useful_stats[data_root][model_name]['i1'][1]))
    useful_stats[data_root][model_name]['i1_corr_diff_per_img'] = [np.zeros(len(model_i1s_concat)), np.zeros(len(model_i1s_concat))]
    for img_index, model_i1_per_img in enumerate(model_i1s_concat) :
            useful_stats[data_root][model_name]['i1_corr_diff_per_img'][0][img_index] = 1 - np.abs(model_i1s_concat[img_index] - human_i1[img_index])
            useful_stats[data_root][model_name]['i1_corr_diff_per_img'][1][img_index] = 1 - np.abs(model_i1s_concat[img_index] - marmoset_i1[img_index])


In [ ]:
compute_upright = True
compute_inverted = True
compute_reverse_contrast = True 

# data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/'
# # data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/'
# bio_i1s_list = [human_i1_5rep, marmoset_i1_5rep]
# bio_i1s_list = [human_i1_5rep]
# bio_i1s_list = [human_i1s_vbsl]
# bio_i1s_list = [human_i1s_obj_vbsl151]

# i1 corr - upright only 
for model_name in model_names_list:
    # get model i1s
    if compute_upright :
        model_i1s = useful_stats[data_root][model_name]['i1_5rep'] 
        model_i1s = subset_upright_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_upright'] = np.zeros((len(bio_i1s_list), 2))
        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_upright_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)

            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][1] = np.std(np.array(i1_corr))

    if compute_inverted : 
        model_i1s = useful_stats[data_root][model_name]['i1_5rep']
        model_i1s = subset_inverted_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_inverted'] = np.zeros((len(bio_i1s_list), 2))

        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_inverted_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4)
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)


            # print(np.mean(np.array(bio_internal)), np.std(np.array(bio_internal)))
    
            useful_stats[data_root][model_name]['i1_corr_inverted'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_inverted'][bio_index][1] = np.std(np.array(i1_corr))

    if compute_reverse_contrast : 
        model_i1s = useful_stats[data_root][model_name]['i1_5rep']
        model_i1s = subset_reverse_contrast_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_reverse_contrast'] = np.zeros((len(bio_i1s_list), 2))
        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_reverse_contrast_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)

            # print(np.mean(np.array(bio_internal)), np.std(np.array(bio_internal)))
    
            useful_stats[data_root][model_name]['i1_corr_reverse_contrast'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_reverse_contrast'][bio_index][1] = np.std(np.array(i1_corr))


# print acc and i1s

In [ ]:
# print acc and i1s
# data_root = '../../data/face_data/vbsl_151/'
# data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/'
# data_root = '/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/control_exp/objs/'

# model_name_list = ['resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_best', 'rn50_preIN_texture_sizeVar_best', # basel finetuned
#                    'rn50_FromScratch_notexture_sizeVar_best', 'rn50_FromScratch_texture_sizeVar_best', # basel FromScratch
#                     'rn50_preIN_notexture_sizeVar_2losses', # DepthMap prediction
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl
# model_name_list = ['vbsl50k_subset_0.5', 'vbsl50k_subset_0.25', 'vbsl50k_subset_0.1', 'vbsl50k_subset_0.05']
# model_name_list = ['Basel_50k_2id',]
# model_name_list = []
# for i in range(41):
#     step = (i+1)*15
#     model_name_list.append(f'vbsl50k_step{step}')
#     # model_name_list.append(f'vbsl50kobj_step{step}')
# model_name_list = ['pixel']
# model_name_list = ['resnet50', 'resnet50_layer1'] # SL
# model_name_list = ["resnet50", "resnet50_layer1", "resnet50_layer2", 
#                     "resnet50_layer3", "resnet50_layer4_no_pooling"]
# model_name_list = ["rn50_preIN_texture_sizeVar_best", "rn50_preIN_notexture_sizeVar_best", "resnet50-trained-pretrained-vbsl", "rn50_vbsl-dist-ft_epoch15"]

acc_list = []
acc_std_list = []


i1_corr_human_list = []
i1_corr_human_list_upright, i1_corr_human_list_inverted, i1_corr_human_list_reverse_contrast = [], [], []

i1_corr_std_human_list = []
i1_corr_std_human_list_upright, i1_corr_std_human_list_inverted, i1_corr_std_human_list_reverse_contrast = [], [], []

i1_corr_AJ_list = []
i1_corr_AJ_list_upright, i1_corr_AJ_list_inverted, i1_corr_AJ_list_reverse_contrast = [], [], []

i1_corr_std_AJ_list = []
i1_corr_std_AJ_list_upright, i1_corr_std_AJ_list_inverted, i1_corr_std_AJ_list_reverse_contrast = [], [], []

for model_name in model_names_list:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_corr_upright = useful_stats[data_root][model_name]['i1_corr_upright']
    i1_corr_inverted = useful_stats[data_root][model_name]['i1_corr_inverted']
    i1_corr_reverse_contrast = useful_stats[data_root][model_name]['i1_corr_reverse_contrast']
    print(i1_corr_reverse_contrast)
    acc = useful_stats[data_root][model_name]['acc']
    acc_list.append(acc[0])
    acc_std_list.append(acc[1])
    # print stats
    print(f"{model_name}")
    print("acc: %.6f±%.6f" % (acc[0],acc[1]))
    for i1_index, cur_i1 in enumerate(i1_corr):
        print(f"i1 #{i1_index}: %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list.append(cur_i1[0])
            i1_corr_std_human_list.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list.append(cur_i1[0])
            i1_corr_std_AJ_list.append(cur_i1[1])
    # ------------------ SL --------------------------
    for i1_index, cur_i1 in enumerate(i1_corr_upright):
        print(f"i1 #{i1_index} (upright): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_upright.append(cur_i1[0])
            i1_corr_std_human_list_upright.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_upright.append(cur_i1[0])
            i1_corr_std_AJ_list_upright.append(cur_i1[1])
    for i1_index, cur_i1 in enumerate(i1_corr_inverted):
        print(f"i1 #{i1_index} (inverted): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_inverted.append(cur_i1[0])
            i1_corr_std_human_list_inverted.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_inverted.append(cur_i1[0])
            i1_corr_std_AJ_list_inverted.append(cur_i1[1])
    for i1_index, cur_i1 in enumerate(i1_corr_reverse_contrast):
        print(f"i1 #{i1_index} (reverse contrast): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_reverse_contrast.append(cur_i1[0])
            i1_corr_std_human_list_reverse_contrast.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_reverse_contrast.append(cur_i1[0])
            i1_corr_std_AJ_list_reverse_contrast.append(cur_i1[1])
    # ------------------------------------------------
       
    # print subset acc
    tmp = useful_stats[data_root][model_name]['acc_per_img'].mean(0)
    if is_print_subset:
        tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
        for subset_id, subset in enumerate(subset_list):
            # get acc
            print("%s acc: %.4f" % (subset_name_list[subset_id], tmp_group[subset].mean()*100))
    print()
    # print(f"{model_name} \n%.2f/%.2f/%.2f" % (i1_corr[0][0], i1_corr[1][0], i1_corr[2][0]))

# plot

In [ ]:
import pickle
with open('useful_stats_SL_20240724_faceobject_face_obj', 'rb') as f:
    useful_stats = pickle.load(f)

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/AppleMesh09/data/face_data/vbsl_151/']['SL_resnet50_finetune_vbsl_50k_face_obj_seed777_model_best_finetune_vbsl_50k_seed777_model_best_finetune_vbsl_50k_obj_seed777_model_best']['acc']

## plot acc

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_1'].keys()

In [ ]:
dataset_names = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_1', '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_2', '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_4', '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_6', '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle151_AppleMesh00/vbsle_AppleMesh00_happiness_8']

In [ ]:
acc_resnet = []
acc_vgg = []
acc_alexnet = []
acc_obj50k = []
acc_face50k = []
acc_4way  = []
            
for dn in dataset_names : 
    acc_resnet.append(useful_stats[dn]['resnet50']['acc'][0])
    acc_vgg.append(useful_stats[dn]['vgg16']['acc'][0])
    acc_alexnet.append(useful_stats[dn]['alexnet']['acc'][0])
    acc_obj50k.append(useful_stats[dn]['SL_resnet50_finetune_vbsl_50k_obj_seed777_model_best']['acc'][0])
    acc_face50k.append(useful_stats[dn]['SL_resnet50_finetune_vbsl_50k_seed77_model_best']['acc'][0])
    acc_4way .append(useful_stats[dn]['SL_resnet50_finetune_vbsl_50k_face_obj_all_seed777_model']['acc'][0])

task_names = ['happiness_1', 'happiness_2', 'happiness_4','happiness_6','happiness_8']
model_names = ['alexnet','vgg16','resnet50',   'obj50k', 'face50k', '4-way']

# Combine all accuracies into a list of lists
all_accuracies = [acc_alexnet, acc_vgg, acc_resnet, acc_obj50k, acc_face50k, acc_4way]

# Plotting
plt.figure(figsize=(11, 11))

# Define colors for each model
colors = ['lightgray','gray', 'darkslategray',  'green', 'blue', 'darkorange']
for i, acc in enumerate(all_accuracies):
    plt.plot(task_names, acc, marker='o', linestyle='-', markersize=12,linewidth=3, color=colors[i], label=model_names[i])

# Adding labels and title
plt.xlabel('Tasks')
plt.ylabel('Accuracy')
plt.title('Model Accuracies on Different Tasks')
plt.legend(title="Models", loc="best")
plt.ylim([0.5, 0.9])
plt.xlabel('Tasks', fontsize=20, labelpad=15)          # Set x-axis label font size
plt.ylabel('Accuracy', fontsize=20, labelpad=15)       # Set y-axis label font size
plt.title('Accuracies on Different Degrees of Happiness', fontsize=25, pad=20)  # Set title font size
plt.xticks(fontsize=15, rotation=0)                   # Set x-axis ticks font size
plt.yticks([0.5, 0.6, 0.7, 0.8, 0.9], fontsize=15)                   # Set y-axis ticks font size

# Adding a legend with a custom font size
plt.legend(loc="best", fontsize='xx-large')

# Display the plot
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import shutil
import json
import pickle
from tqdm import tqdm
import random
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image

from load_model import load_model
from rn50_auxiliary_dm import rn50_auxiliary_dm

import scipy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [ ]:
# Define custom x positions for the tasks (e.g., 0, 1, 3, 5, 7 for uneven spacing)
x_positions = [0, 5, 15, 25, 35]

# Plotting
plt.figure(figsize=(9, 10))

# Define colors for each model
colors = ['lightgray', 'gray', 'darkslategray', 'orange', 'green', 'blue']
for i, acc in enumerate(all_accuracies):
    plt.plot(x_positions, acc, marker='o', linestyle='-', markersize=9, linewidth=2.5, color=colors[i], label=model_names[i])

# Adding labels and title
plt.xlabel('Tasks')
plt.ylabel('Accuracy')
plt.title('Model Accuracies on Different Tasks')
plt.legend(title="Models", loc="best")

# Set custom x-ticks and labels
plt.xticks(x_positions, task_names, fontsize=15, rotation=0)

# Set y-axis limits and ticks
plt.ylim([0.5, 0.9])
plt.yticks([0.5, 0.6, 0.7, 0.8, 0.9], fontsize=15)

# Adjust font sizes
plt.xlabel('Tasks', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)
plt.title('Model Accuracies on Different Tasks', fontsize=23)

# Adding a legend with a custom font size
plt.legend(title="Models", loc="best", fontsize='xx-large')

# Display the plot
plt.grid(True)
plt.show()

In [ ]:
task_names = ['happiness_1', 'happiness_2', 'happiness_4','happiness_6','happiness_8']
model_names = ['resnet50', 'vgg16', 'alexnet', 'obj 50k', 'face 50k', '4-way']

In [ ]:
# acc - GFR
models = ["resnet50", "resnet50-layer1", "resnet50-layer2", "resnet50-layer3", "resnet50-layer4-no-pooling"]
models = ["finetuned-Basel", "finetuned-Basel-NoTexture", "finetuned-AppleFaces", "finetuned-AppleFaces-dist"]

models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["face50k", "cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask", "4-way"]
models = ["face50k", "cat-face50k", "cat-obj-face50k", "multitask", "4-way"]
# models = []

# for model_name in model_names_list :
#     models.append(model_name.split("-")[-1])

plt.figure(figsize=(8, 8))  
acc_list = [n / 100 for n in [57.8589, 60.0405, 85.8958]]
acc_list = [n / 100 for n in [85.895, 61.1054, 60.2877, 83.2418, 82.7832,  68.6334, 77.0670]]

acc_std_list = [n / 100 for n in [2.497, 2.7894, 1.5592]]
# acc_std_list = [n / 100 for n in [1.55, 2.9224, 2.7483, 1.8827, 1.6929,  2.5420, 1.8957]]
bars = plt.bar(models, acc_list, yerr=acc_std_list, capsize=5)

# Add text annotations for each bar
for bar, acc in zip(bars, acc_list):
    yval = bar.get_height()
    bar.set_color('lightblue')

    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)
# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models],rotation=30, fontsize=18)
plt.yticks(fontsize=15)

plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("accuracy", fontsize=20)

# Set the range of y-axis from 0 to 1
plt.ylim(0.0, 1.05)
plt.axhline(y=57.8589/100, color='gray', linestyle='--', linewidth=2)
# bars[-1].set_color('orange')
# bars.set_color('lightblue')
plt.legend()
plt.title("ResNet50 from scratch, accuracy", fontsize=20)
plt.title("GFR vbsl151 AppleMesh00-AppleMesh01", fontsize=20)
plt.show()

In [ ]:
# acc - GOR
models = ["resnet50", "resnet50-layer1", "resnet50-layer2", "resnet50-layer3", "resnet50-layer4-no-pooling"]
models = ["finetuned-Basel", "finetuned-Basel-NoTexture", "finetuned-AppleFaces", "finetuned-AppleFaces-dist"]

models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["face50k", "cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]

# models = []

# for model_name in model_names_list :
#     models.append(model_name.split("-")[-1])

plt.figure(figsize=(7, 8))  
acc_list = [n / 100 for n in [72.5794, 85.6704, 69.7722]]
# acc_list = [n / 100 for n in [69.7722, 72.1811, 83.5643, 67.9867, 77.4176, 84.3719, 82.3081]]

acc_std_list = [n / 100 for n in [2.2267, 1.4935, 2.2411]]
# acc_std_list = [n / 100 for n in [2.24, 2.3768, 1.8118, 2.5082, 2.0191, 1.5362, 1.9147]]
bars = plt.bar(models, acc_list, yerr=acc_std_list, capsize=5)

# Add text annotations for each bar
for bar, acc in zip(bars, acc_list):
    yval = bar.get_height()
    bar.set_color("lightblue")
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)
# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models],rotation=30, fontsize=18)
plt.yticks(fontsize=15)

plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("accuracy", fontsize=20)

# Set the range of y-axis from 0 to 1
plt.ylim(0.0, 1.05)
plt.axhline(y=72.5794/100, color='gray', linestyle='--', linewidth=2)
# bars[-1].set_color('orange')
# bars[0].set_color('lightblue')

plt.legend()
plt.title("ResNet50 from scratch, accuracy", fontsize=20)
plt.title("GOR vbsl151 camel-elephant", fontsize=20)
plt.show()

In [ ]:
a = 1

In [ ]:
# acc - AppleMesh03-praneeth]
import matplotlib.pyplot as plt
models = ["resnet50", "resnet50-layer1", "resnet50-layer2", "resnet50-layer3", "resnet50-layer4-no-pooling"]
models = ["finetuned-Basel", "finetuned-Basel-NoTexture", "finetuned-AppleFaces", "finetuned-AppleFaces-dist"]

models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask", "4-way"]

# models = []

# for model_name in model_names_list :
#     models.append(model_name.split("-")[-1])

plt.figure(figsize=(9, 8))  
acc_list = [n / 100 for n in [69.7722,77.4176, 84.3719, 82.308,     86.7465]]
acc_list = [0.356536, 0.383913,0.427112 ,0.380207, 0.422545]
acc_std_list = [0.023619,0.015610,0.023217,0.021730,0.024793]
bars = plt.bar(models3, acc_list, yerr=acc_std_list, capsize=5)

# Add text annotations for each bar
for bar, acc in zip(bars, acc_list):
    bar.set_color('dimgray')
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)
# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)

plt.xticks(ticks=np.arange(len(models3)), labels=[label.replace('-', '-\n') for label in models3],rotation=30, fontsize=18)
plt.yticks(fontsize=15)

plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=20)

# Set the range of y-axis from 0 to 1
plt.ylim(-0.05, 1.05)
plt.axhline(y=0.4181, color='gray', linestyle='--', linewidth=2)
bars[-1].set_color('orange')
bars[-2].set_color('orange')

bars[0].set_color('lightblue')

plt.legend()
plt.title("ResNet50 from scratch, accuracy", fontsize=20)
plt.title("GFR vbsl151 AppleMesh00-AppleMesh01, i1", fontsize=20)
plt.title("GOR vbsl151 camel-elephant, i1", fontsize=20)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]# Dummy data for demonstration purposes
# Define the position of the bars
x = np.arange(len(models))  # the label locations
width = 0.2  # the width of the bars
acc_list = [n / 100 for n in [72.5794,85.6704,69.77]]

fig, ax = plt.subplots(figsize=(9, 8))
acc_list_upright = [val / 100 for val in [75.24,97.57,71.23]]
                    # [71.23,97.57,74.38]]
acc_list_inverted = [val / 100 for val in [71.79, 83.6 ,70.83]]
    # [70.83,83.6,71.2]]
acc_list_reverse_contrast = [val / 100 for val in [70.56,75.61, 67.55]]
# [67.55,75.61,70.22]]

# Plotting each set of bars
bars0 = ax.bar(x - width * 1.5, acc_list, width,label='all', color='blue', alpha=1.0, capsize=5)
bars1 = ax.bar(x - width * 0.5 , acc_list_upright, width,label='upright', color='blue', alpha=0.65, capsize=5)
bars2 = ax.bar(x +  width * 0.5, acc_list_inverted, width,  label='inverted', color='blue', alpha=0.3, capsize=5)
bars3 = ax.bar(x + width * 1.5, acc_list_reverse_contrast, width, label='reverse contrast', color='blue', alpha=0.1, capsize=5)

# Add text annotations for each bar
for bars in [bars0, bars1, bars2, bars3]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=15)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('models', fontsize=18)
ax.set_ylabel('accuracy', fontsize=18)
ax.set_title('GOR vbsl151 camel-elephant', fontsize=22)
ax.set_xticks(x)
ax.set_xticklabels([label.replace('-', '-\n') for label in models], rotation=30,fontsize=18)
ax.yaxis.set_tick_params(labelsize=15)

ax.legend(fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')


# Display the plot
plt.ylim(0.0, 1.05)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]
# Define the position of the bars
x = np.arange(len(models3))  # the label locations
width = 0.2  # the width of the bars
acc_list = [n / 100 for n in [85.8958, 82.7832, 68.6334, 77.0670]]

fig, ax = plt.subplots(figsize=(11, 8))
acc_list_upright = [val / 100 for val in [98.65	, 98.3,83.73, 96.9]]
                    # [71.23,97.57,74.38]]
acc_list_inverted = [val / 100 for val in [76.39, 75.76, 	63.48, 	66.32]]
    # [70.83,83.6,71.2]]
acc_list_reverse_contrast = [val / 100 for val in [	82.37,74, 	58.17	, 	67.79]]
# [67.55,75.61,70.22]]

# Plotting each set of bars
bars0 = ax.bar(x - width * 1.5, acc_list, width,label='all', color='blue', alpha=1.0, capsize=5)
bars1 = ax.bar(x - width * 0.5 , acc_list_upright, width,label='upright', color='blue', alpha=0.65, capsize=5)
bars2 = ax.bar(x +  width * 0.5, acc_list_inverted, width,  label='inverted', color='blue', alpha=0.3, capsize=5)
bars3 = ax.bar(x + width * 1.5, acc_list_reverse_contrast, width, label='reverse contrast', color='blue', alpha=0.1, capsize=5)

# Add text annotations for each bar
for bars in [bars0, bars1, bars2, bars3]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=15)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('models', fontsize=18)
ax.set_ylabel('accuracy', fontsize=18)
ax.set_title('GFR vbsl151 AppleMesh00-AppleMesh01', fontsize=22)
ax.set_xticks(x)
ax.set_xticklabels([label.replace('-', '-\n') for label in models3], rotation=30,fontsize=18)
ax.yaxis.set_tick_params(labelsize=15)

ax.legend(fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')


# Display the plot
plt.ylim(0.0, 1.05)
plt.show()


In [ ]:
# SL

acc_list_upright = [val/100 for val in [98.65,65.91]]
acc_std_list_upright = []
acc_list_inverted = [val/100 for val in [76.39,61.16]]
acc_std_list_inverted = []
acc_list_reverse_contrast = [val/100 for val in [82.37,52.9]]
acc_std_list_reverse_contrast = []


plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_upright, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_upright):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)

plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50 (upright only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_inverted, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_inverted):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50 (inverted only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_reverse_contrast, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_reverse_contrast):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, (reverse contrast only)", fontsize=20)
plt.show()

## plot i1s

In [ ]:
# SL
models = ["resnet", "obj50k", "face50k"]
models2 = ["cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k"]
models3 = ["face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]

plt.figure(figsize=(7, 8)) 
i1_corr_human_list = [-0.000972,0.157688, 0.485140]
i1_corr_std_human_list = [0.002378, 0.016036, 0.006226]
# [0.023619, 0.020221, 0.023948]
bars = plt.bar(models, i1_corr_human_list, yerr=i1_corr_std_human_list, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list):
    yval = bar.get_height()
    bar.set_color('dimgray')
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=15)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=30, fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(-0.05, 1.05)

plt.axhline(y=-0.000972, color='gray', linestyle='--', linewidth=2)

plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("GFR vbsl151 AppleMesh00-AppleMesh01, i1", fontsize=20)
plt.show()

# ---------------------------- SL -------------------------------------
plt.figure(figsize=(7, 8)) 
i1_corr_human_list = [0.418159,0.398955, 0.356536]
i1_corr_std_human_list = [0.021829, 0.020221, 0.023619]
# [0.023619, 0.020221, 0.023948]
bars = plt.bar(models, i1_corr_human_list, yerr=i1_corr_std_human_list, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list):
    yval = bar.get_height()
    bar.set_color('dimgray')
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=15)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=30, fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(-0.05, 1.05)
plt.axhline(y=0.418159, color='gray', linestyle='--', linewidth=2)

plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("GOR vbsl151 camel-elephant, i1", fontsize=20)
plt.show()

In [ ]:
# acc - AppleMesh03-praneeth
models = ["resnet50", "resnet50-layer1", "resnet50-layer2", "resnet50-layer3", "resnet50-layer4-no-pooling"]
models = ["finetuned-Basel", "finetuned-Basel-NoTexture", "finetuned-AppleFaces", "finetuned-AppleFaces-dist"]

models = ["resnet", "obj50k", "face50k"]
models2 = ["face50k","cat50k", "cat-obj50k", "cat-face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]
models3 = ["face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]

# models = []

# for model_name in model_names_list :
#     models.append(model_name.split("-")[-1])

plt.figure(figsize=(11, 8))  
acc_list = [0.485140, 0.004014,0.070360,0.442748,0.517701,0.365679, 0.538343]
# acc_list = [0.356536, 0.355510,0.427566,0.304946,0.383913,0.427112,0.380207]
acc_std_list = [0.006226, 0.017121 ,0.005082 ,0.013043 ,0.015055 ,0.010541, 0.011796]
# acc_std_list = [0.0236190, 0.011726 ,0.017798 ,0.029011 ,0.015610 ,0.023217, 0.021730]



bars = plt.bar(models2, acc_list, yerr=acc_std_list, capsize=5)

# Add text annotations for each bar
for bar, acc in zip(bars, acc_list):
    yval = bar.get_height()
    bar.set_color('dimgray')
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)
# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)

plt.xticks(ticks=np.arange(len(models2)), labels=[label.replace('-', '-\n') for label in models2],rotation=30, fontsize=18)
plt.yticks(fontsize=15)

plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=20)

# Set the range of y-axis from 0 to 1
plt.ylim(-0.05, 1.05)
plt.axhline(y=-0.000972, color='gray', linestyle='--', linewidth=2)
bars[-1].set_color('orange')
bars[0].set_color('lightblue')

plt.legend()
plt.title("ResNet50 from scratch, accuracy", fontsize=20)
plt.title("GFR vbsl151 AppleMesh00-AppleMesh01, i1", fontsize=20)
plt.show()

In [ ]:
# SL
models3 = ["face50k", "cat-obj-face50k", "cat-face-obj50k", "multitask"]

i1_corr_human_list_upright = [0.191045, 0.512367, 0.361097, 0.421580]
# [0.424810,0.512017,0.347575]
i1_corr_std_human_list_upright = [0.472004,0.392035,0.493194]
# [0.035555, 0.030237, 0.031485]
i1_corr_human_list_inverted = [0.28614,0.250046,0.335397,0.454984]
# [0.035555, 0.030237, 0.031485]
i1_corr_std_human_list_inverted = [0.015472,0.037951,0.019945]
# [0.034771,0.033141, 0.029989]
i1_corr_human_list_reverse_contrast = [0.365510, 0.361112  , 0.08786, 0.318961]
i1_corr_std_human_list_reverse_contrast = [0.007847, 0.017272, 0.011585]
# [0.010393,0.007654,0.014628]

plt.figure(figsize=(6, 8)) 

bars = plt.bar(models3, i1_corr_human_list_upright, yerr=i1_corr_std_human_list_upright, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_upright):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=15)
plt.xticks(ticks=np.arange(len(models3)), labels=[label.replace('-', '-\n') for label in models3], fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GOR\nvbsli*151, (upright only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models3, i1_corr_human_list_inverted, yerr=i1_corr_std_human_list_inverted, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_inverted):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

plt.xticks(ticks=np.arange(len(models3)), labels=[label.replace('-', '-\n') for label in models3], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GFR\nvbsli*151, (inverted only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models3, i1_corr_human_list_reverse_contrast, yerr=i1_corr_std_human_list_reverse_contrast, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_reverse_contrast):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

plt.xticks(ticks=np.arange(len(models3)), labels=[label.replace('-', '-\n') for label in models3], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GFR\nvbsli*151, (reverse contrast only)", fontsize=20)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
models = ["finetuned-AppleFaces50k", "finetuned-Object50k", "vAppleMesh10lla resnet"]
i1_corr_human_list = [0.485140, 0.517701   , 0.365679, 0.538343]
# Define the position of the bars
x = np.arange(len(models3))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(11, 8))

# Plotting each set of bars
bars0 = ax.bar(x - width * 1.5, i1_corr_human_list, width,label='all', color='green', alpha=1.0, capsize=5)
bars1 = ax.bar(x - width * 0.5 , i1_corr_human_list_upright, width,label='upright', color='green', alpha=0.65, capsize=5)
bars2 = ax.bar(x +  width * 0.5, i1_corr_human_list_inverted, width,  label='inverted', color='green', alpha=0.3, capsize=5)
bars3 = ax.bar(x + width * 1.5, i1_corr_human_list_reverse_contrast, width, label='reverse contrast', color='green', alpha=0.1, capsize=5)

# Add text annotations for each bar
for bars in [bars0, bars1, bars2, bars3]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=18)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('models', fontsize=18)
ax.set_ylabel('i1', fontsize=18)
ax.set_title('Finetuned ResNet50, tested on GOR vbsli*151', fontsize=22)
ax.set_xticks(x)

ax.set_xticklabels([label.replace('-', '-\n') for label in models3], rotation=30,fontsize=18)
ax.yaxis.set_tick_params(labelsize=15)

ax.legend(fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')


# Display the plot
plt.ylim(-0.05, 1.05)
plt.show()


## plot category effect

In [ ]:
# obj50k vs cat-obj50k
models = ['face50k', 'cat-face50k']
tasks = ['GOR 151', 'GFR 151', 'ood AppleMesh03-praneeth', 'ood Basel-texture', 'ood Basel-no texture', 'GOR i1', 'GFR i1']

plt.figure(figsize=(13, 8))  
acc_list1 = [n/100 for n in [69.7722, 85.8958, 68.0398, 74.0292, 74.9414, 0.356536 * 100, 0.485140 * 100]]
acc_list2 = [n/100 for n in [67.9867, 83.2418, 66.9641, 71.5211, 76.2519, 0.304946 * 100, 0.442748* 100]]
acc_std_list1 = [n/100 for n in [2.2411, 1.5592, 2.1830, 2.299, 2.11786, 0.023619 * 100, 0.006226 * 100]]
acc_std_list2 = [n/100 for n in [2.5082, 1.8827, 1.9066, 2.0418, 1.7854, 0.029011 * 100, 0.013043 * 100]]

# Positioning of bars
bar_width = 0.35
x = np.arange(len(tasks))

bars1 = plt.bar(x - bar_width/2, acc_list1, bar_width, yerr=acc_std_list1, capsize=5, label=models[0], color='lightblue')
bars2 = plt.bar(x + bar_width/2, acc_list2, bar_width, yerr=acc_std_list2, capsize=5, label=models[1])

# change color of the last four bars to grey
for bar in bars1[-2:]:
    bar.set_color('lightgrey')
for bar in bars2[-2:]:
    bar.set_color('grey')

# add text annotations
for bar, acc in zip(bars1, acc_list1):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.5, yval + 0.02, f'{acc:.2f}', ha='center', va='bottom', fontsize=12)

for bar, acc in zip(bars2, acc_list2):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.5, yval + 0.02, f'{acc:.2f}', ha='center', va='bottom', fontsize=12)

plt.xticks(ticks=x, labels=[label.replace('-', '-\n') for label in tasks], rotation=30, fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("task", fontsize=20)
plt.ylabel("accuracy", fontsize=20)
plt.ylim(0,1)
plt.legend(models, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=15)

# add a second y-axis
ax2 = plt.gca().twinx()
ax2.set_ylabel("i1", fontsize=20)
ax2.yaxis.set_ticks([])

# Set the range of y-axis from 0 to 1
# plt.axhline(y=-0.000972, color='gray', linestyle='--', linewidth=2)

# ax2.legend(models, bbox_to_anchor=(1.05, 1), loc='upper right', fontsize=15)

plt.title("Comparing models w/ & w/o category training", fontsize=20)
plt.show()

In [ ]:
# obj50k vs cat-obj50k
models = ['face-obj50k', 'cat-face-obj50k']
tasks = ['GOR 151', 'GFR 151', 'ood AppleMesh03-praneeth', 'ood Basel-texture', 'ood Basel-no texture', 'GOR i1', 'GFR i1']

plt.figure(figsize=(13, 8))  
acc_list1 = [n/100 for n in [87.1642, 68.0978 ,59.0477, 69.2275,	73.1752 ,0.450757 *100,0.239189*100]]
acc_list2 = [n/100 for n in [84.3719, 68.6334 ,57.9432, 71.0934,	69.3325 ,0.427112*100, 0.244190*100]]
acc_std_list1 = [n/100 for n in [ 1.6079,  2.4525 , 2.3252,  2.7898 , 2.0581 , 0.022016*100  ,0.008785*100]]
acc_std_list2 = [n/100 for n in [ 1.5362  ,2.5420,  2.4381,  2.2071 , 2.358 , 0.023217*100  ,0.016437*100]]

# Positioning of bars
bar_width = 0.35
x = np.arange(len(tasks))

bars1 = plt.bar(x - bar_width/2, acc_list1, bar_width, yerr=acc_std_list1, capsize=5, label=models[0], color='lightblue')
bars2 = plt.bar(x + bar_width/2, acc_list2, bar_width, yerr=acc_std_list2, capsize=5, label=models[1])

# change color of the last four bars to grey
for bar in bars1[-2:]:
    bar.set_color('lightgrey')
for bar in bars2[-2:]:
    bar.set_color('grey')

# add text annotations
for bar, acc in zip(bars1, acc_list1):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.5, yval + 0.02, f'{acc:.2f}', ha='center', va='bottom', fontsize=12)

for bar, acc in zip(bars2, acc_list2):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.5, yval + 0.02, f'{acc:.2f}', ha='center', va='bottom', fontsize=12)

plt.xticks(ticks=x, labels=[label.replace('-', '-\n') for label in tasks], rotation=30, fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("task", fontsize=20)
plt.ylabel("accuracy", fontsize=20)
plt.ylim(0,1)
plt.legend(models, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=15)

# add a second y-axis
ax2 = plt.gca().twinx()
ax2.set_ylabel("i1", fontsize=20)
ax2.yaxis.set_ticks([])

# Set the range of y-axis from 0 to 1
# plt.axhline(y=-0.000972, color='gray', linestyle='--', linewidth=2)

# ax2.legend(models, bbox_to_anchor=(1.05, 1), loc='upper right', fontsize=15)

plt.title("Comparing models w/ & w/o category training", fontsize=20)
plt.show()